In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import re
from tqdm import tqdm
import numpy as np
import pandas as pd
import nltk

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/samplecoded4.csv",encoding="utf-8")
dataa = pd.read_csv("/content/drive/MyDrive/round1c.csv",encoding="utf-8")
dataaa = pd.read_csv("/content/drive/MyDrive/round2fixc.csv",encoding="utf-8")

dataa = dataa[["ref_id","text_o","effectiveness","risiko","distribution","mandatory","politics","pharma","CT","Child"]]
dataaa = dataaa[["ref_id","text_o","effectiveness","risiko","distribution","mandatory","politics","pharma","CT","Child"]]

data = pd.concat([data, dataa],axis=0,ignore_index=True, sort=False)
data = pd.concat([data, dataaa],axis=0,ignore_index=True, sort=False)


# Remove links
http_link_pattern = r'http\S+'
bitly_link_pattern = r'bit.ly/\S+'
data['cleaned'] = data['text_o'].str.replace(http_link_pattern, '')
data['cleaned'] = data['text_o'].str.replace(bitly_link_pattern, '')

import random
random.seed(999)
codelist =  ["effectiveness","risiko","distribution","mandatory","politics","pharma","CT","Child"]
cate = "mandatory"

data1 = data[data[cate].map(lambda x: x == 1)].reset_index()
data0 = data[data[cate].map(lambda x: x == 0)].reset_index()

samplelen = len(data1)
ranlist = random.sample(range(len(data0)), samplelen)
data0 = data0.iloc[ranlist,:]
texts = np.array(data0["cleaned"].values.tolist()+data1["cleaned"].values.tolist())
labels = np.array(data0[cate].values.tolist()+data1[cate].values.tolist())
#print(texts)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: FutureWarning: The default value of regex will change from True to False in a future version.
  app.launch_new_instance()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: FutureWarning: The default value of regex will change from True to False in a future version.


In [ ]:
from sklearn.model_selection import train_test_split
sentences = texts
y = labels
sentences_train, sentences_test, y_train, y_test = train_test_split(sentences, y, test_size=0.10, random_state=1000)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
vectorizer.fit(sentences_train)
X_train = vectorizer.transform(sentences_train)
X_test  = vectorizer.transform(sentences_test)

In [ ]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier.fit(X_train, y_train)
score = classifier.score(X_test, y_test)
print("Accuracy:", score)

Accuracy: 0.8695652173913043


In [ ]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 100) 
 
# Training the model on the training dataset
# fit function is used to train the model using the training sets as parameters
classifier.fit(X_train, y_train)
 
# performing predictions on the test dataset
y_pred = classifier.predict(X_test)
 
# metrics are used to find accuracy or error
from sklearn import metrics 
print()
 
# using metrics module for accuracy calculation
print("ACCURACY OF THE MODEL: ", metrics.accuracy_score(y_test, y_pred))


ACCURACY OF THE MODEL:  0.8641304347826086


In [ ]:
from sklearn.svm import SVC
  
# Building a Support Vector Machine on train data
svc_model = SVC(C= .1, kernel='linear', gamma= 1)
svc_model.fit(X_train, y_train)
  
prediction = svc_model .predict(X_test)
# check the accuracy on the training set
#print(svc_model.score(X_train, y_train))
print(svc_model.score(X_test, y_test))
from sklearn.metrics import confusion_matrix
print("Confusion Matrix:\n",confusion_matrix(prediction,y_test))

0.8641304347826086
Confusion Matrix:
 [[75 14]
 [11 84]]
